In [2]:
# Dependencies and Setup
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import re
import time 
import datetime 
import numpy as np
from config1 import username1,password1,host1,port1,database1
from config2 import username2,password2,host2,port2,database2
import sqlalchemy
from sqlalchemy import create_engine, func, inspect, desc
pd.options.mode.chained_assignment = None  # default='warn'
import requests
import json
from pandas.io.json import json_normalize

In [3]:
# Database Setup
connection1=f'{username1}:{password1}@{host1}:{port1}/{database1}'
engine1 = create_engine(f'postgresql://{connection1}')

# Pull Data
state_heatmap=pd.read_sql_query('select * from state_heatmap', con=engine1)
usa_heatmap=pd.read_sql_query('select * from usa_heatmap', con=engine1)
county_heatmap=pd.read_sql_query('select * from county_heatmap', con=engine1)
orders=pd.read_sql_query('select * from orders', con=engine1)
state_cases=pd.read_sql_query('select * from state_cases', con=engine1)
state_deaths=pd.read_sql_query('select * from state_deaths', con=engine1)
county_cases1=pd.read_sql_query('select * from county_cases1', con=engine1)
county_cases2=pd.read_sql_query('select * from county_cases2', con=engine1)
county_cases3=pd.read_sql_query('select * from county_cases3', con=engine1)
county_deaths1=pd.read_sql_query('select * from county_deaths1', con=engine1)
county_deaths2=pd.read_sql_query('select * from county_deaths2', con=engine1)
county_deaths3=pd.read_sql_query('select * from county_deaths3', con=engine1)

# Concats
county_cases=pd.concat([county_cases1,county_cases2,county_cases3], ignore_index=True)
county_deaths=pd.concat([county_deaths1,county_deaths2,county_deaths3], ignore_index=True)

In [4]:
state_heatmap

,state,order_date,order_expiration_date,population,total_cases,total_deaths,thisweek_cases,lastweek_cases,thisweek_deaths,lastweek_deaths,per100k,positive_rate
0,Alabama,2020-04-03,2020-04-30,4903185,92661,1633,11546,12224,142,200,1889.812438,12.976804
1,Alaska,2020-03-27,2020-04-24,731545,3341,25,718,673,4,3,456.704646,1.653330
2,Arizona,2020-03-30,2020-05-15,7278717,179497,3779,15670,18644,475,520,2466.052740,18.611950
3,Arkansas,None,None,3017804,44597,475,5150,5520,67,45,1477.796437,8.456011
4,California,2020-03-19,None,39512223,516851,9441,58730,60251,947,724,1308.078768,6.291022
5,Colorado,2020-03-25,2020-04-26,5758736,47950,1844,3403,3999,45,41,832.647998,8.719838
6,Connecticut,2020-03-20,2020-05-20,3565287,50062,4437,1079,928,19,12,1404.150634,6.037516
7,Delaware,2020-03-22,2020-05-31,973764,15055,585,649,782,6,56,1546.062496,8.022530
8,District Of Columbia,2020-03-30,2020-05-29,705749,12313,586,455,519,4,3,1744.671264,6.193195
9,Florida,2020-04-01,2020-05-04,21477737,491884,7157,59137,72353,1226,859,2290.204038,13.107127


In [5]:
usa_heatmap

,population,total_cases,total_deaths,thisweek_cases,lastweek_cases,thisweek_deaths,lastweek_deaths,positive_rate,per100k,lastdate
0,339191516,4712528,155384,423096,455568,7389,6986,8.157476,1389.341354,8/3/20


In [6]:
county_heatmap

,fips,county,state,total_cases,total_deaths,thisweek_cases,lastweek_cases,thisweek_deaths,lastweek_deaths,population,per100k
0,72001,Adjuntas,Puerto Rico,86,0,9,5,0,0,19483,441.410460
1,72003,Aguada,Puerto Rico,108,0,16,21,0,0,41959,257.394123
2,72005,Aguadilla,Puerto Rico,192,0,36,44,0,0,60949,315.017474
3,72007,Aguas Buenas,Puerto Rico,114,0,20,13,0,0,28659,397.780802
4,72009,Aibonito,Puerto Rico,79,0,9,10,0,0,25900,305.019305
...,...,...,...,...,...,...,...,...,...,...,...
3215,56037,Sweetwater,Wyoming,250,0,29,40,0,0,42343,590.416362
3216,56039,Teton,Wyoming,358,0,66,69,0,0,23464,1525.741562
3217,56041,Uinta,Wyoming,265,0,30,13,0,0,20226,1310.194799
3218,56043,Washakie,Wyoming,49,0,4,1,0,0,7805,627.802691


In [7]:
orders

,state,order_date,order_expiration_date
0,Alabama,2020-04-03,2020-04-30
1,Alaska,2020-03-27,None
2,Arizona,2020-03-30,2020-05-15
3,Arkansas,None,None
4,California,2020-03-19,None
5,Colorado,2020-03-25,2020-04-26
6,Connecticut,2020-03-20,2020-05-20
7,Delaware,2020-03-22,2020-05-31
8,District of Columbia,2020-03-30,2020-05-29
9,Florida,2020-04-01,2020-05-04


In [8]:
state_cases

,State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20
0,Alabama,0,0,0,0,0,0,0,0,0,...,71813,74212,76005,78130,79294,81115,82366,83782,85762,87723
1,Alaska,0,0,0,0,0,0,0,0,0,...,2131,2193,2249,2339,2525,2623,2730,2798,2879,2990
2,Arizona,0,0,0,0,1,1,1,1,1,...,150609,152944,156301,160041,162014,163827,165934,168273,170798,174010
3,Arkansas,0,0,0,0,0,0,0,0,0,...,35246,36259,37249,37981,38623,39447,40181,40968,41759,42511
4,California,0,0,0,0,2,2,2,2,2,...,421286,430773,440185,446452,452288,458121,470762,484913,492934,500556
5,Colorado,0,0,0,0,0,0,0,0,0,...,41680,42295,42962,43771,44318,44547,45296,45778,46186,46791
6,Connecticut,0,0,0,0,0,0,0,0,0,...,48223,48232,48776,48776,48776,48983,49077,49540,49670,49810
7,Delaware,0,0,0,0,0,0,0,0,0,...,13792,13924,14202,14175,14290,14406,14476,14602,14689,14788
8,District of Columbia,0,0,0,0,0,0,0,0,0,...,11529,11571,11649,11717,11780,11858,11945,11999,12057,12057
9,Florida,0,0,0,0,0,0,0,0,0,...,379619,389868,402312,414511,423855,432747,441977,451423,461379,470386


In [9]:
state_deaths

,State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20
0,Alabama,0,0,0,0,0,0,0,0,0,...,1364,1397,1438,1456,1473,1491,1491,1538,1565,1580
1,Alaska,0,0,0,0,0,0,0,0,0,...,19,19,19,20,20,21,22,22,23,23
2,Arizona,0,0,0,0,0,0,0,0,0,...,2974,3063,3143,3286,3305,3304,3408,3454,3626,3694
3,Arkansas,0,0,0,0,0,0,0,0,0,...,380,386,394,399,401,408,428,434,442,453
4,California,0,0,0,0,0,0,0,0,0,...,8047,8201,8337,8408,8448,8494,8679,8908,9026,9214
5,Colorado,0,0,0,0,0,0,0,0,0,...,1771,1786,1790,1794,1794,1799,1807,1822,1822,1838
6,Connecticut,0,0,0,0,0,0,0,0,0,...,4406,4410,4413,4413,4413,4418,4423,4425,4431,4432
7,Delaware,0,0,0,0,0,0,0,0,0,...,527,529,578,579,579,579,580,581,581,585
8,District of Columbia,0,0,0,0,0,0,0,0,0,...,580,581,581,581,581,582,583,584,584,584
9,Florida,0,0,0,0,0,0,0,0,0,...,5345,5518,5653,5777,5854,5931,6117,6333,6586,6843


In [10]:
county_cases

,FIPS,County,State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20
0,21007,Ballard,Kentucky,0,0,0,0,0,0,0,...,28,28,28,28,29,30,30,NaN,NaN,NaN
1,21009,Barren,Kentucky,0,0,0,0,0,0,0,...,247,250,262,276,290,302,315,NaN,NaN,NaN
2,21011,Bath,Kentucky,0,0,0,0,0,0,0,...,27,28,29,31,31,32,32,NaN,NaN,NaN
3,21013,Bell,Kentucky,0,0,0,0,0,0,0,...,221,230,234,239,260,262,267,NaN,NaN,NaN
4,21015,Boone,Kentucky,0,0,0,0,0,0,0,...,922,929,934,946,957,965,984,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141,56037,Sweetwater,Wyoming,0,0,0,0,0,0,0,...,214,215,221,228,234,237,240,NaN,NaN,NaN
2142,56039,Teton,Wyoming,0,0,0,0,0,0,0,...,275,279,292,311,318,331,335,NaN,NaN,NaN
2143,56041,Uinta,Wyoming,0,0,0,0,0,0,0,...,232,233,235,243,245,247,254,NaN,NaN,NaN
2144,56043,Washakie,Wyoming,0,0,0,0,0,0,0,...,45,45,45,45,45,46,48,NaN,NaN,NaN


In [11]:
county_deaths

,FIPS,County,State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20
0,72001,Adjuntas,Puerto Rico,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,72003,Aguada,Puerto Rico,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,72005,Aguadilla,Puerto Rico,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,72007,Aguas Buenas,Puerto Rico,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,72009,Aibonito,Puerto Rico,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,56037,Sweetwater,Wyoming,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3216,56039,Teton,Wyoming,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3217,56041,Uinta,Wyoming,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3218,56043,Washakie,Wyoming,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
